# Machine-Learning Interatomic Potentials

Running simulations with energies and forces from *ab initio* methods, such as DFT allows for higher accuracy and the investigation of different systems. E.g., *ab initio* MD can be used to study chemical reactions on an atomistic scale. With the recent advancments in machine-learning interatomic potentials (MLIPs) it is possible to scale up these types of simulations to large systems [allegro paper] and long simulation times [bmim paper] while retaining the accuracy of the underlying *ab initio* methods.

In this tutorial you will use pre-trained MLIPs to run simulations within ESPResSo. There are many different MLIPs architectures and respective open-source python packages:
- MACE
- NequIP / allegro
- Apax
- DeepMD
- TorchANI

In this tutorial, we will focus on the MACE-MP-0 model, which has been trained on the materials project dataset and is able to accuratly predict energies and forces for many organic and inorganic systems.

Most MLIPs provide a Python interface through the ASE package. ESPResSo also utilizes this interface, and thus we will have a quick overview of the ASE features. The main object used is the `ase.Atoms`, which represents a single frame of an atomistic simulation.

We can use 

In [6]:
import espressomd
import ipsuite as ips
import numpy as np
import pandas as pd
import pint
import plotly.express as px
import tqdm
from espressomd.plugins.ase import ASEInterface
from espressomd.zn import Visualizer
from mace.calculators import mace_mp
from rdkit2ase import smiles2atoms

For MLIP the typical unit system is energies in eV, distances in Angstrom and time in fs and mass in atomic units. This is also the default unit system within ASE. We will use pint to ensure correct unit usage within espresso.

In [7]:
ureg = pint.UnitRegistry()
boltmk = 1.380649e-23 * ureg.J / ureg.K
boltmk = boltmk.to(ureg.eV / ureg.K)

In this tutorial we will use SMILES representations and rdkit to generate starting structures. In this first part, we will look at a gas phase system of Ethanol, run a geometry optimization followed by a short MD simulation. First, let us instantiate our MLIP.

In [8]:
mace_mp_calc = mace_mp()

Using Materials Project MACE for MACECalculator with /tikhome/fzills/.cache/mace/5yyxdm76
Using float32 for MACECalculator, which is faster but less accurate. Recommended for MD. Use float64 for geometry optimization.


/data/fzills/miniconda3/envs/espresso/lib/python3.10/site-packages/mace/calculators/mace.py:128: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_l

Default dtype float32 does not match model dtype float64, converting models to float32.


We can now create our ASE atoms object and attach the aforementioned MLIP calculator to it. 

In [9]:
etoh = smiles2atoms("CCO")
etoh.calc = mace_mp_calc
etoh

Atoms(symbols='C2OH6', pbc=False, calculator=MACECalculator(...))

We can now levarage the ase interface to compute energies and forces.

In [10]:
print(etoh.get_potential_energy() * ureg.eV)
print(etoh.get_forces() * ureg.eV / ureg.angstrom)

-47.11187744140625 electron_volt
[[-0.24490514397621155 0.0379762202501297 0.03378382325172424] [-0.27195578813552856 0.8330878019332886 0.14494556188583374] [1.3262779712677002 0.09577012062072754 -0.1421809196472168] [0.10953038930892944 0.20156031847000122 0.4934889078140259] [0.000935770571231842 0.327559232711792 -0.4345357418060303] [0.2787896990776062 -0.5502724051475525 -0.1073441356420517] [-0.06919784098863602 -0.2714897096157074 0.2776922285556793] [-0.13868796825408936 -0.1908503770828247 -0.3161454498767853] [-0.9907872676849365 -0.48334163427352905 0.050295814871788025]] electron_volt / angstrom


Now that we know how to compute the properties that interest us, it is time to plug it into espresso. We will create a box of arbitrary size, define a simulation time step of 0.5 fs to resolve the movement of the hydrogens and run a geometry optimization. For MLIP a good starting structure is still very important, because e.g. overlapping atoms can lead the models to produce entirely non-physical structures and "explode", because they often do not include any Lennard-Jones or other short ranged repulsion potentials but purely rely on the output of the neural network.

In [11]:
system = espressomd.System(box_l=[16] * 3)
system.time_step = (
    (0.05 * ureg.fs)
    .to(((1 * ureg.u * ureg.angstrom**2) / ureg.electron_volt) ** 0.5)
    .magnitude
)
system.cell_system.skin = 0.4

We add the ASE structure to the espresso system and use the provided ASEInterface to go back to ASE, such that we can use the calculator to compute forces.

In [12]:
for atom in etoh:
    system.part.add(pos=atom.position, type=atom.number, mass=atom.mass)
system.ase = ASEInterface(type_mapping={x: x for x in set(etoh.numbers)})
system.ase.get()

Atoms(symbols='C2OH6', pbc=True, cell=[16.0, 16.0, 16.0], calculator=SinglePointCalculator(...))

We will use the ZnDraw visualizer integrated with espresso to follow the simulation. To see the effect of the geometry optimization, we display the atomic forces.

In [13]:
vis = Visualizer(system)
vis.zndraw.config.scene.vectors = "forces"
vis.zndraw.config.scene.simulation_box = False
vis

Starting ZnDraw server, this may take a few seconds


Showing ZnDraw at http://127.0.0.1:1234/token/5bc89467


In [14]:
system.integrator.set_steepest_descent(f_max=0.1, gamma=4, max_displacement=0.001)

In [15]:
tbar = tqdm.trange(500, ncols=120)
for idx in tbar:
    forces = mace_mp_calc.get_forces(system.ase.get())
    system.part.all().ext_force = forces
    system.integrator.run(1)
    vis.zndraw.append(system.ase.get())
    tbar.set_description(f"fmax: {np.linalg.norm(forces, axis=0).max():.3f}")

  0%|                                                                                           | 0/500 [00:00<?, ?it/s]

fmax: 0.126: 100%|████████████████████████████████████████████████████████████████████| 500/500 [00:15<00:00, 31.93it/s]


In this way, we can get ground state energies using MLIP and espresso. You can investigate different systems by altering the smiles representation.

### MD Simulations of Sulfuric Acid and Water

What makes MLIP so attractive beyond their *ab initio* energy and force calculation accuracy, is the ability to describe chemical reactions. Observing chemical reactions in MD simulations requires overcoming the energy of the reaction barrier. Methods that aid this process are plentiful but beyond the scope of this tutorial. Therefore, we will look at the proton transfer from sulfuric acid to water which can be seen without biasing the simulation.

In [16]:
# clear the system for a fresh start
system.part.clear()
system.thermostat.turn_off()

In this part of the tutorial, we generate two different molecular species - water and sulfuric acid - and use packmol to create a suitable starting structure. We will use the Python interface to packmol provided within IPSuite. IPSuite provides many tools used around MLIP training, evaluation and deployment.

In [17]:
water = smiles2atoms("O")
sulfuric_acid = smiles2atoms("OS(=O)(=O)O")
packmol = ips.nodes.Packmol(
    data=[[water], [sulfuric_acid]], count=[100, 1], density=1000
)

In [18]:
packmol.run()

2024-08-16 09:05:46,094 (INFO): estimated box size: [14.665660145847443, 14.665660145847443, 14.665660145847443]


INFO:ipsuite.configuration_generation.packmol:estimated box size: [14.665660145847443, 14.665660145847443, 14.665660145847443]



################################################################################

 PACKMOL - Packing optimization for the automated generation of
 starting configurations for molecular dynamics simulations.
 
                                                              Version 20.14.0 

################################################################################

  Packmol must be run with: packmol < inputfile.inp 

  Userguide at: http://m3g.iqm.unicamp.br/packmol 

  Reading input file... (Control-C aborts)
  Seed for random number generator:      1234567
  Output file: mixture.xyz
  Reading coordinate file: 0.xyz
  Reading coordinate file: 1.xyz
  Number of independent structures:            2
  The structures are: 
  Structure            1 :Properties=species:S:1:pos:R:3 pbc="F F F"(           3  atoms)
  Structure            2 :Properties=species:S:1:pos:R:3 pbc="F F F"(           7  atoms)
  Maximum number of GENCAN loops for all molecule packing:          400
  Total numbe

We will repeat the geometry optimization process. Now that we have a bulk structure, we need to set the correct cell vectors, which we also get from the ase structure we just created.

In [19]:
system.box_l = packmol.atoms[0].get_cell().diagonal()
system.time_step = (
    (0.05 * ureg.fs)
    .to(((1 * ureg.u * ureg.angstrom**2) / ureg.electron_volt) ** 0.5)
    .magnitude
)
system.cell_system.skin = 0.4

In [20]:
system.ase = ASEInterface(type_mapping={x: x for x in set(packmol.atoms[0].numbers)})
for atom in packmol.atoms[0]:
    system.part.add(pos=atom.position, type=atom.number, mass=atom.mass)
system.integrator.set_steepest_descent(f_max=0.1, gamma=4, max_displacement=0.001)

We will reset ZnDraw and enable showing the box this time.

In [21]:
del vis.zndraw[:]
vis.zndraw.config.scene.simulation_box = True
vis

In [22]:
tbar = tqdm.trange(500, ncols=120)
for _ in tbar:
    atoms = system.ase.get()
    atoms.calc = mace_mp_calc
    system.part.all().ext_force = atoms.get_forces()
    vis.zndraw.append(atoms)
    system.integrator.run(1)
    tbar.set_description(
        f"fmax: {np.linalg.norm(atoms.get_forces(), axis=0).max():.3f}"
    )

  0%|                                                                                           | 0/500 [00:00<?, ?it/s]

fmax: 1.810: 100%|████████████████████████████████████████████████████████████████████| 500/500 [01:07<00:00,  7.44it/s]


Wit this minimized structure, we can now run an MD simulation.
Let us highlight the two hydrogen atoms from the sulfuric acid, These are the particles we want to follow.

In [23]:
vis.zndraw.selection = [305, 306]

In [24]:
# Langevin dynamics at 400 K
system.integrator.set_vv()
system.thermostat.set_langevin(kT=(400 * ureg.K * boltmk).magnitude, gamma=2, seed=42)

In [27]:
tbar = tqdm.trange(5000, ncols=120)
for idx in tbar:
    atoms = system.ase.get()
    atoms.calc = mace_mp_calc
    system.part.all().ext_force = atoms.get_forces()
    if idx % 5 == 0:
        vis.zndraw.append(atoms)
    system.integrator.run(1)
    tbar.set_description(f"e_pot: {atoms.get_potential_energy():.3f} eV")

e_pot: -1485.783: 100%|█████████████████████████████████████████████████████████████| 5000/5000 [09:38<00:00,  8.64it/s]


We have visually observed the reaction of the hydrogen atoms. Let us now plot the distance from the hydrogens to sulfur from the sulfate ion we have generated.

In [ ]:
OH_1 = []
OH_2 = []
for atoms in vis.zndraw:
    OH_1.append(atoms.get_distance(300, 305, mic=True))
    OH_2.append(atoms.get_distance(304, 306, mic=True))

df = pd.DataFrame({"OH_1": OH_1, "OH_2": OH_2})
fig = px.line(df, y=["OH_1", "OH_2"])

We can attach the figure to the visualizer, such that we can see the trajectory and the distance side-by-side.

In [ ]:
vis.zndraw.figure = fig.to_json()

## References

- Batatia, I.; Benner, P.; Chiang, Y.; Elena, A. M.; Kovács, D. P.; Riebesell, J.; Advincula, X. R.; Asta, M.; Baldwin, W. J.; Bernstein, N.; Bhowmik, A.; Blau, S. M.; Cărare, V.; Darby, J. P.; De, S.; Della Pia, F.; Deringer, V. L.; Elijošius, R.; El-Machachi, Z.; Fako, E.; Ferrari, A. C.; Genreith-Schriever, A.; George, J.; Goodall, R. E. A.; Grey, C. P.; Han, S.; Handley, W.; Heenen, H. H.; Hermansson, K.; Holm, C.; Jaafar, J.; Hofmann, S.; Jakob, K. S.; Jung, H.; Kapil, V.; Kaplan, A. D.; Karimitari, N.; Kroupa, N.; Kullgren, J.; Kuner, M. C.; Kuryla, D.; Liepuoniute, G.; Margraf, J. T.; Magdău, I.-B.; Michaelides, A.; Moore, J. H.; Naik, A. A.; Niblett, S. P.; Norwood, S. W.; O’Neill, N.; Ortner, C.; Persson, K. A.; Reuter, K.; Rosen, A. S.; Schaaf, L. L.; Schran, C.; Sivonxay, E.; Stenczel, T. K.; Svahn, V.; Sutton, C.; van der Oord, C.; Varga-Umbrich, E.; Vegge, T.; Vondrák, M.; Wang, Y.; Witt, W. C.; Zills, F.; Csányi, G. A Foundation Model for Atomistic Materials Chemistry. arXiv December 29, 2023. https://doi.org/10.48550/arXiv.2401.00096.
- Batatia, I.; Kovacs, D. P.; Simm, G.; Ortner, C.; Csanyi, G. MACE: Higher Order Equivariant Message Passing Neural Networks for Fast and Accurate Force Fields. In Advances in neural information processing systems; Koyejo, S., Mohamed, S., Agarwal, A., Belgrave, D., Cho, K., Oh, A., Eds.; Curran Associates, Inc., 2022; Vol. 35, pp 11423–11436. https://openreview.net/forum?id=YPpSngE-ZU.
- Landrum, G.; Tosco, P.; Kelley, B.; Ric; Cosgrove, D.; sriniker; gedeck; Vianello, R.; NadineSchneider; Kawashima, E.; N, D.; Jones, G.; Dalke, A.; Cole, B.; Swain, M.; Turk, S.; AlexanderSavelyev; Vaucher, A.; Wójcikowski, M.; Take, I.; Probst, D.; Ujihara, K.; Scalfani, V. F.; godin,  guillaume; Lehtivarjo, J.; Pahl, A.; Walker, R.; Berenger, F.; jasondbiggs; strets123. Rdkit/Rdkit: 2023_03_2 (Q1 2023) Release, 2023. https://doi.org/10.5281/zenodo.8053810.
- Martínez, L.; Andrade, R.; Birgin, E. G.; Martínez, J. M. PACKMOL: A Package for Building Initial Configurations for Molecular Dynamics Simulations. J Comput Chem 2009, 30 (13), 2157–2164. https://doi.org/10.1002/jcc.21224.
- Zills, F.; Schäfer, M. R.; Segreto, N.; Kästner, J.; Holm, C.; Tovey, S. Collaboration on Machine-Learned Potentials with IPSuite: A Modular Framework for Learning-on-the-Fly. J. Phys. Chem. B 2024. https://doi.org/10.1021/acs.jpcb.3c07187.